In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
bike2019 =pd.read_csv('../input/ford-gobike-2019feb-tripdata/201902-fordgobike-tripdata.csv')

# 1.Data extraction

In [ ]:
bike2019.head()

In [ ]:
bike2019.describe()

In [ ]:
bike2019.info()

# 2 Cleaning Process

At the beggining lets take a look at the duplicates and null values.

In [ ]:
bike2019['start_time']=pd.to_datetime(bike2019['start_time'])
bike2019['end_time']=pd.to_datetime(bike2019['end_time'])

In [ ]:
bike2019['start_time'].duplicated().value_counts()

In [ ]:
bike2019[bike2019['start_time'].duplicated()]

In [ ]:
bike2019[bike2019.duplicated(subset=['end_time'])]

Taking a look at the start and end time duplicates we can see that they are totaly different rides. We dont hace to take any 
action. 

### Null values


In [ ]:
bike2019.dropna(inplace=True)

In [ ]:
bike2019.info()

### Quality and tidness issues

1. I will make a column caled  'Customer_age' in ordere to make the data more user friendly.
2. I will transform the ride duration fron seconds to minutes.
3. I will make a new categorical column named 'general_runtime', seperating the ride duration at 'casual' and 'long' rides.
4. I will make a new categorical column named 'Customer_age' , seperating the ages by decate.
5. I will separete the start_date and 3 months. "hour_start", 'day_start','month_start' to help our analysis. 



## 1.

In [ ]:
bike2019['Customer_age']=2019-bike2019['member_birth_year']
bike2019=bike2019.drop('member_birth_year',axis=1)

## 2.

In [ ]:
bike2019['duration_sec']=bike2019['duration_sec']/60
bike2019=bike2019.rename(columns={'duration_sec':'duration_min'})

## 3.

In [ ]:
bin_edges=[bike2019['duration_min'].min(),bike2019['duration_min'].mean(),bike2019['duration_min'].max()]
bin_names=['casual','long']
bike2019['general_runtime']=pd.cut(bike2019['duration_min'],bin_edges,labels=bin_names)

## 4.

In [ ]:
bin_edges=[0,18.000000,30,40,50,60,70,140]
bin_names=['underage','18s','30s','40s','50s','60s','>70']
bike2019['Customer_decade']=pd.cut(bike2019['Customer_age'],bin_edges,labels=bin_names)

## 5.

In [ ]:
bike2019['hour_start']=bike2019['start_time'].dt.hour

In [ ]:
bike2019['day_start']=bike2019['start_time'].dt.weekday_name

In [ ]:
bike2019['month_start']=bike2019['start_time'].dt.month

In [ ]:
bike2019=bike2019.drop('start_time',axis=1)
bike2019=bike2019.drop('end_time',axis=1)

**6.**


In [ ]:
bike2019['hour_start'].describe()

In [ ]:
bin_edges=[0,6,12,16,19,23]
bin_names=['aftermidnigt','morning','midday','afternoon','night']
bike2019['day_period']=pd.cut(bike2019['hour_start'],bin_edges,labels=bin_names)

## Test

In [ ]:
bike2019.head()

In [ ]:
bike2019.to_csv('clean_bikes.csv')

# 3 Exploratory analysis

In [ ]:
bike2019=pd.read_csv('clean_bikes.csv')

In [ ]:
bike2019.hist(figsize=(9,8));

In [ ]:
bike2019.info()

## The structure of the dataset
There are 174951 bike rides in the dataset with 19 features . Most variables are categorical.

## The main features of interest in my dataset.

I'm most interested in figuring out how does every different biker group (gender, age, custormer-subscriber) behave and how the bike use change through time.

I'll start by looking at the distribution of the main variable of interest: duration

In [ ]:
# there's a long tail in the distribution, so let's put it on a log scale instead
log_binsize = 0.025
bins = 10 ** np.arange(2.4, np.log10(bike2019['duration_min'].max())+log_binsize, log_binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = bike2019, x = 'duration_min', bins = bins)
plt.xscale('log')

plt.xlabel('duration (min)')
plt.show()

Duration has a long-tailed distribution, with a lot of rides on the short duration end, and few on the long duration end. When plotted on a log-scale, the duration distribution keeps the same shape.

I want to check the duration distribution for the shortest rides

In [ ]:
a=bike2019[bike2019['duration_min']<60]
a.hist(['duration_min'],bins = 1000);

* We can see that the majority of the rides are between 3-5 mins

In [ ]:
# let's plot all some categorical variables together to get an idea of each  variable's distribution.

fig, ax = plt.subplots(nrows=2, figsize = [8,8])

default_color = sb.color_palette()[0]
sb.countplot(data = bike2019, x = 'Customer_decade', color = default_color, ax = ax[0])
sb.countplot(data = bike2019, x = 'member_gender', color = default_color, ax = ax[1])

plt.show()

The bikers are generaly young with the majority in their 20s-30s. Additionaly the majority are men.

In [ ]:
sb.regplot(data=a, x='duration_min',y='Customer_age',scatter_kws={'alpha': 1/20})

In [ ]:
plt.figure(figsize = [8, 8])

# subplot 1: color vs cut
plt.subplot(2, 1, 1)
sb.countplot(data = bike2019, x = 'member_gender', hue = 'Customer_decade', palette = 'Blues')

# subplot 2: clarity vs. cut
ax = plt.subplot(2, 1, 2)
sb.countplot(data = bike2019, x = 'hour_start', hue = 'member_gender', palette = 'Greens')
ax.legend(ncol = 2) # re-arrange legend to reduce overlapping



plt.show()

We can see that the majority of riders are at decades 20 and 30.

In [ ]:
a=bike2019[bike2019['duration_min']<60]
base_color = sb.color_palette()[3]
sb.boxplot(data = a, x='duration_min', y='member_gender', color = base_color);

What is intresting is that the Females have the biggest duration mean on their rides. At the other side, men taking the shortest rides. 

Lets see the same diagramm only for the younger riders.

In [ ]:
y=a[a['Customer_age']<30]
base_color = sb.color_palette()[3]
sb.boxplot(data = y, x='duration_min', y='member_gender', color = base_color);

Here the differences between the genders are bigger.

We can see the same thing below:

In [ ]:
young=bike2019[bike2019['Customer_age']<30]

gender = [['Male', '^'],['Female','o'],
               ['Other', 's']]

for gen, marker in gender:
    df_gender = young[young['member_gender'] == gen]
    plt.scatter(data = df_gender, x = 'duration_min', y = 'Customer_age',
                marker = marker,alpha = 1/5)
    
plt.legend(['Male','Female','Other']);

We can see that the summer months have the longest rides and the October have the more rides.

In [ ]:
bike2019['duration_min'].describe(),bike2019['Customer_age'].describe(),bike2019['month_start'].describe()

In [ ]:
ccounts=bike2019.groupby(['member_gender','Customer_decade']).size()
ccounts = ccounts.reset_index(name = 'count')
ccounts = ccounts.pivot(index = 'Customer_decade', columns = 'member_gender', values = 'count')

The biggest group is "Male riders at their 20s" and the smallest is "Other riders older than 70 years old"

In [ ]:
dcounts=bike2019.groupby(['month_start','day_start']).size()
dcounts = dcounts.reset_index(name = 'count')
dcounts = dcounts.pivot(index = 'day_start', columns = 'month_start', values = 'count')

In [ ]:
dcounts

In [ ]:
sb.heatmap(dcounts)

* The most crowded days where the Thuesdays and Mondays of October.

In [ ]:
hcounts=bike2019.groupby(['day_start','hour_start']).size()
hcounts = hcounts.reset_index(name = 'count')
hcounts = hcounts.pivot(index = 'hour_start', columns = 'day_start', values = 'count')

In [ ]:
plt.figure(figsize=(14, 7))
b=sb.heatmap(hcounts,square=False)
b.set_title('Bike usage per hour')
b.set_ylabel('Hour')
b.set_xlabel('Day');

We can see that the more rides are on working days at 8:00 and 16:00-17:00. This can lead us to the conclusion that the riders are using their bikes to go to work.  

In [ ]:
plt.figure(figsize=(14, 7))
mcount=bike2019.groupby('hour_start').size()
mcount = mcount.reset_index(name = 'count')
g=sb.barplot(data = mcount, x = 'hour_start',y='count');
g.set_title('Rides per hour')
g.set_ylabel('Rides')
g.set_xlabel('Hour');

Now lets see the movement per day. I want to take a look how the daily usage is separated on the kind of the client (Subscriber or Customer) and gender.

In [ ]:
plt.figure(figsize=(16, 6))
a=bike2019[bike2019['duration_min']<=60]
g=sb.violinplot(x=a['day_start'],y=a['duration_min'],hue='user_type',data=a, order=["Monday", "Tuesday","Wednesday","Thursday",
                                                                                    "Friday","Saturday","Sunday"],scale="width",cut=0)
g.set_title('Duration per day')
g.set_ylabel('Duration (min)')
g.set_xlabel('Day')
plt.xticks(rotation=90);

We can see that 'Customers' are taking longer rides every day. Generally at weekends the rides are longer for all genders.

4.What are the riding habbits of subrcibers and ordinary customers?

It is obvious that at weekends the rides are generally longer especially for the Customers. The subscribers seem to keep the same duration mean for every day in a week. I can assume that this is happening because subscribers are using the bikes for a specific reason and no for leisure. That is why we do not see longer rides at the weekend by them. At the other hand, the Customers are using the bikes mainly at weekends.

In [ ]:
plt.figure(figsize=(14, 6))
b=sb.countplot(data = bike2019, x = 'day_start',hue='user_type',order=["Monday", "Tuesday","Wednesday","Thursday",
                                                                                  "Friday","Saturday","Sunday"]);
b.set_title('Rides amount per month seperated by rides length')
b.set_ylabel('Rides')
b.set_xlabel('Day')
plt.xticks(rotation=90)
plt.show();

In [ ]:
plt.figure(figsize=(16, 6))
a=bike2019[bike2019['duration_min']<=60]
g=sb.violinplot(x=a['day_start'],y=a['duration_min'],hue='member_gender',order=["Monday", "Tuesday","Wednesday","Thursday",
                                                                                  "Friday","Saturday","Sunday"],data=a)
plt.xticks(rotation=90)
g.set_title('Duration per day separated by gender')
g.set_ylabel('Duration (min)')
g.set_xlabel('Day');

In [ ]:
mcount=bike2019.groupby('start_station_name').size().sort_values().tail()
mcount = mcount.reset_index(name = 'count')
pl=sb.barplot(data = mcount, x ='start_station_name',y='count');
pl.set_title('Most popular start stations')
pl.set_ylabel('Rides')
pl.set_xlabel('Station name');
plt.xticks(rotation=90);

In [ ]:
mcount=bike2019.groupby('end_station_name').size().sort_values().tail()
mcount = mcount.reset_index(name = 'count')
pl=sb.barplot(data = mcount, x ='end_station_name',y='count');
pl.set_title('Most popular destination stations')
pl.set_ylabel('Rides')
pl.set_xlabel('Station name');
plt.xticks(rotation=90);

**Duration Prediction**
Lets try to predict the duration of a ride,of course without knowing the destination of the rider.

In [ ]:
X=bike2019.copy()

In [ ]:
#https://www.ncdc.noaa.gov/cdo-web/quickdata#

In [ ]:
X.drop(['Unnamed: 0', 'start_station_name','end_station_id','end_station_name', 'end_station_latitude', 'end_station_longitude','general_runtime'], axis=1,inplace=True)

In [ ]:
X=X[X['user_type']=='Subscriber']

In [ ]:
X.info()

In [ ]:
X=pd.get_dummies(X)

In [ ]:
corr = X.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


In [ ]:
# Remove rows with missing target, separate target from predictors

y = X.duration_min              
X.drop(['duration_min'], axis=1, inplace=True)

# Break off validation set from training data

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.9, test_size=0.1, random_state=1)

In [ ]:
X_train.info(),X_test.info()

In [ ]:
y_val

In [ ]:
def score_dataset(XT, XV, YT, YV,n,r):
    model= XGBRegressor(n_estimators=n,verbosity =0,learning_rate=0.1)
    model.fit(XT, YT, 
             early_stopping_rounds=r, 
             eval_set=[(XV, YV)], 
             verbose=False)
    preds = model.predict(XV)
    mae = mean_absolute_error(YV, preds)
    return mae

In [ ]:
stopping_rounds=[5,10,50,100]
estimators=[100,500,1000,2000]
learning_rate=[0.1,0.3,0.5,0.7,1]

In [ ]:
#imputed_results = pd.DataFrame(columns = ['stopping_rounds','estimators','MAE'])

#for n in estimators: 
#    for r in stopping_rounds: 
#        m=score_dataset(X_train, X_val, y_train, y_val,n,r)
#        imputed_results = imputed_results.append({"stopping_rounds":r,"estimators":n,"MAE":m},ignore_index=True)

In [ ]:
#imputed_results.sort_values(by=['MAE'])

In [ ]:
model= XGBRegressor(n_estimators=500,verbosity =0,learning_rate=0.1)
model.fit(X_train, y_train, 
        early_stopping_rounds=50, 
          eval_set=[(X_val, y_val)],verbose=False)
preds = model.predict(X_val)
mae = mean_absolute_error(y_val, preds)
print(mae)

In [ ]:
model= XGBRegressor(n_estimators=500,verbosity =0,learning_rate=0.1)
model.fit(X_train, y_train, 
             early_stopping_rounds=50, 
             eval_set=[(X_val, y_val)],verbose=False)
    
final_predictions=model.predict(X_test)
#final_predictions=round(final_predictions,2)
#y_test=round(y_test,2)
X_test['prediction']=final_predictions.astype('float64')
X_test['actual_duration']=y_test

In [ ]:
X_test.info()

In [ ]:
labels=X_test[15800:].index
pred = X_test[15800:]['prediction']
act_dur = X_test[15800:]['actual_duration']

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(100,20))
rects1 = ax.bar(x - width/2, pred, width, label='prediction')
rects2 = ax.bar(x + width/2, act_dur, width, label='actual_duration')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Duration')
ax.set_title('Actual and predicted ride duration')
ax.set_xticks(x)

ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()


